In [1]:
import pandas as pd

In [2]:
#load csv file as pandas dataframe where first row is not column name
df = pd.read_csv('home240_bedroom2228_sensor14307c14311_electric-mains_electric-combined.csv', header=None)
df.head()

,0,1
0,2017-11-06 11:11:59,0
1,2017-11-06 11:12:00,146
2,2017-11-06 11:12:01,148
3,2017-11-06 11:12:02,148
4,2017-11-06 11:12:03,148


In [5]:
#define column names
df.columns = ['datetime', 'power (watts)']

In [6]:
#convert datetime column to datetime object
df['datetime'] = pd.to_datetime(df['datetime'])

In [7]:
df['datetime'].head(1)

0   2017-11-06 11:11:59
Name: datetime, dtype: datetime64[ns]

In [8]:
df['datetime'][17923599] - df['datetime'][0]

Timedelta('236 days 12:48:00')

In [9]:
df

,datetime,power (watts)
0,2017-11-06 11:11:59,0
1,2017-11-06 11:12:00,146
2,2017-11-06 11:12:01,148
3,2017-11-06 11:12:02,148
4,2017-11-06 11:12:03,148
...,...,...
17923595,2018-06-30 23:59:55,121
17923596,2018-06-30 23:59:56,121
17923597,2018-06-30 23:59:57,121
17923598,2018-06-30 23:59:58,121


In [10]:
df_consp = df.copy().reset_index(drop=True, inplace=False)
df_consp

,datetime,power (watts)
0,2017-11-06 11:11:59,0
1,2017-11-06 11:12:00,146
2,2017-11-06 11:12:01,148
3,2017-11-06 11:12:02,148
4,2017-11-06 11:12:03,148
...,...,...
17923595,2018-06-30 23:59:55,121
17923596,2018-06-30 23:59:56,121
17923597,2018-06-30 23:59:57,121
17923598,2018-06-30 23:59:58,121


In [11]:
df_consp['time (s)'] = df_consp['datetime'].diff().dt.seconds
df_consp.head()

,datetime,power (watts),time (s)
0,2017-11-06 11:11:59,0,NaN
1,2017-11-06 11:12:00,146,1.0
2,2017-11-06 11:12:01,148,1.0
3,2017-11-06 11:12:02,148,1.0
4,2017-11-06 11:12:03,148,1.0


In [12]:
# calculate energy consumption in watt-hours
df_consp['Consumption (kWh)'] = df_consp['power (watts)'] * df_consp['time (s)'] / (3600*1000)
df_consp.head()

,datetime,power (watts),time (s),Consumption (kWh)
0,2017-11-06 11:11:59,0,NaN,NaN
1,2017-11-06 11:12:00,146,1.0,0.000041
2,2017-11-06 11:12:01,148,1.0,0.000041
3,2017-11-06 11:12:02,148,1.0,0.000041
4,2017-11-06 11:12:03,148,1.0,0.000041


In [13]:
# sum the energy consumption column only on daily basis in new dataframe
df_daily = df_consp.resample('D', on='datetime').sum()
df_daily.head()

,power (watts),time (s),Consumption (kWh)
datetime,,,
2017-11-06,12020754,46080.0,3.372141
2017-11-07,18337738,86400.0,5.167830
2017-11-08,21071370,86400.0,5.963114
2017-11-09,13785543,86400.0,3.904826
2017-11-10,23783031,86400.0,6.705481


In [14]:
# Only save the rows where time is 24 hours
df_daily = df_daily[df_daily['time (s)']==86400]

In [15]:
df_daily

,power (watts),time (s),Consumption (kWh)
datetime,,,
2017-11-07,18337738,86400.0,5.167830
2017-11-08,21071370,86400.0,5.963114
2017-11-09,13785543,86400.0,3.904826
2017-11-10,23783031,86400.0,6.705481
2017-11-11,20811735,86400.0,5.855660
...,...,...,...
2018-06-26,11129982,86400.0,3.179170
2018-06-27,9069826,86400.0,2.588826
2018-06-28,15059109,86400.0,4.267058


In [16]:
df_daily.drop(['power (watts)'], axis=1, inplace=True)
df_daily.drop(['time (s)'], axis=1, inplace=True)

In [17]:
df_daily['homeid'] = 240

In [18]:
df_daily.reset_index(inplace=True)
df_daily

,datetime,Consumption (kWh),homeid
0,2017-11-07,5.167830,240
1,2017-11-08,5.963114,240
2,2017-11-09,3.904826,240
3,2017-11-10,6.705481,240
4,2017-11-11,5.855660,240
...,...,...,...
203,2018-06-26,3.179170,240
204,2018-06-27,2.588826,240
205,2018-06-28,4.267058,240
206,2018-06-29,4.166689,240


In [19]:
# export dataframe to csv file
df_daily.to_csv('home240_electricity.csv', index=False)